In [1]:
# Importamos los módulos necesarios
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.schema import Document
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.vectorstores import FAISS

/Users/mhurben/anaconda3/envs/proy/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Cargar el PDF de forma asíncrona
file_path = "../01_data/U0924996.pdf"
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [3]:
# Crear un text splitter que divida el texto en chunks de hasta 512 tokens
# Puedes ajustar el solapamiento (chunk_overlap) según lo requieras para preservar el contexto
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

# Cargar el tokenizador y el modelo
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Función para contar tokens usando el tokenizador de Hugging Face
def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text, add_special_tokens=False))

# Crear un divisor de texto basado en la cuenta de tokens
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,  # Número máximo de tokens por fragmento
    chunk_overlap=50,  # Superposición opcional entre fragmentos
    length_function=count_tokens,  # Usa el tokenizador de Hugging Face para contar tokens
)


In [4]:
# Procesar cada página y dividirla en fragmentos (chunks)
chunks = []
for page in pages:
    # Se divide el contenido de la página en fragmentos de hasta 512 tokens
    splitted_texts = text_splitter.split_text(page.page_content) # UNA LISTA DE STRINGS

    for chunk_text in splitted_texts:
        # Creamos un objeto Document para cada fragmento, manteniendo la metadata (por ejemplo, número de página)
        chunks.append(Document(page_content=chunk_text, metadata={"page": page.metadata.get("page", "unknown")}))

In [5]:
chunks[0]

Document(metadata={'page': 0}, page_content='I. DISPOSICIONES GENERALES\nMINISTERIO DE UNIVERSIDADES\n17045 Real Decreto 889/2022, de 18 de octubre, por el que se establecen las \ncondiciones y los procedimientos de homologación, de declaración de \nequivalencia y de convalidación de enseñanzas universitarias de sistemas \neducativos extranjeros y por el que se regula el procedimiento para \nestablecer la correspondencia al nivel del Marco Español de Cualificaciones \npara la Educación Superior de los títulos universitarios oficiales \npertenecientes a ordenaciones académicas anteriores.\nI\nEl proceso de construcción de Europa en los últimos años ha estado \nindefectiblemente unido a un crecimiento de la movilidad del estudiantado y del \nprofesorado universitario, como, igualmente, de las personas trabajadoras y \nprofesionales. A medida que se han afianzado las dinámicas de globalización estas se')

In [6]:
# Generar embeddings para cada fragmento utilizando el modelo HuggingFace (LaBSE)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")

vector_store_faiss = FAISS.from_documents(chunks, embeddings)
vector_store_faiss.save_local("../01_data/faiss_index")

vector_store_faiss_loaded = FAISS.load_local("../01_data/faiss_index", embeddings, allow_dangerous_deserialization = True)


# Realizar una búsqueda de similitud para obtener los chunks más relevantes
query = "¿Qué condiciones y limitaciones establece el Real Decreto 1325/2003 para que las personas beneficiarias del régimen de protección temporal sustituyan la documentación exigida por una declaración responsable en el inicio del procedimiento?"
retrieved_docs_with_scores = vector_store_faiss_loaded.similarity_search_with_score(query, k=2)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
contexto = retrieved_docs_with_scores[0][0].page_content
score = retrieved_docs_with_scores[0][1]

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

# Cargar el tokenizador y el modelo
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [01:14<00:00, 37.43s/it]


In [12]:
prompt = f"Resuelve este consulta en Español usando la información explicada en el fragmento Contexto: {contexto} Pregunta: {query}"

In [13]:
import ollama

response = ollama.chat(model="deepseek-r1:14b", messages=[{"role": "user", "content": prompt}])
print(response["message"]["content"])

<think>
Ok, así que tengo que resolver una pregunta sobre un real decreto español, específicamente el Real Decreto 1325/2003. La pregunta es: ¿Qué condiciones y limitaciones establece el Real Decreto 1325/2003 para que las personas beneficiarias del régimen de protección temporal sustituyan la documentación exigida por una declaración responsable en el inicio del procedimiento?

Primero, necesito entender bien lo que se me está preguntando. La persona quiere saber cuáles son las condiciones y limitaciones que establece dicho real decreto para que ciertas personas puedan usar una declaración responsable en lugar de la documentación requerida al iniciar un procedimiento.

Del contexto proporcionado, veo que el Real Decreto habla de las "Personas beneficiarias del régimen de protección temporal". En particular, menciona los artículos 2 y 11 del Reglamento sobre régimen de protección temporal en caso de afluencia masiva de personas desplazadas. También dice que estas personas pueden sustit